In [18]:
#!/usr/bin/env python3
import numpy as np
import os
import tools

# 定义数据所在的目录（请根据实际情况修改）
DATA_DIR = r"D:\Documents\Pycharm_Files\Techtile_Channel_Measurement\samples"

def compute_phi(filename):
    """
    加载指定文件的 IQ 数据，并计算两个通道之间的平均相位差。

    参数:
      filename: 存储 IQ 数据的 .npy 文件路径，数据格式应为 (2, N)。

    返回:
      phi: 计算得到的平均相位差（单位：弧度）。
    """
    iq_samples = np.load(filename)
    # 对两个通道分别提取相位（忽略返回的频率斜率）
    phase_ch0, _ = tools.get_phases_and_apply_bandpass(iq_samples[0, :])
    phase_ch1, _ = tools.get_phases_and_apply_bandpass(iq_samples[1, :])

    # 计算两个通道之间的相位差，并归一化到 [-π, π]
    phase_diff = tools.to_min_pi_plus_pi(phase_ch0 - phase_ch1, deg=False)
    phi = np.mean(phase_diff)
    return phi

def process_device(device, unique_id, data_dir):
    """
    针对指定设备（例如 "A05"）和唯一标识符，处理该设备的 pilot 与 loopback 数据文件，
    计算相应的 φ 值，并返回格式化后的结果字符串。
    """
    # 构造完整文件路径
    pilot_files = [
        os.path.join(data_dir, f"data_{device}_{unique_id}_1_pilot.npy"),
        os.path.join(data_dir, f"data_{device}_{unique_id}_2_pilot.npy")
    ]
    loopback_files = [
        os.path.join(data_dir, f"data_{device}_{unique_id}_1_loopback.npy"),
        os.path.join(data_dir, f"data_{device}_{unique_id}_2_loopback.npy")
    ]

    lines = []
    lines.append(f"=== Device {device} Pilot Signal Measurements ===")
    for i, filepath in enumerate(pilot_files, start=1):
        if not os.path.exists(filepath):
            lines.append(f"File {filepath} not found. Please check the path.")
        else:
            phi_P = compute_phi(filepath)
            lines.append(f"Measurement {i} - Pilot phi_P: {phi_P:.6f} rad")

    lines.append("")
    lines.append(f"=== Device {device} Loopback Signal Measurements ===")
    for i, filepath in enumerate(loopback_files, start=1):
        if not os.path.exists(filepath):
            lines.append(f"File {filepath} not found. Please check the path.")
        else:
            phi_LB = compute_phi(filepath)
            lines.append(f"Measurement {i} - Loopback phi_LB: {phi_LB:.6f} rad")

    lines.append("-" * 50)
    return "\n".join(lines)

def main():
    # 如果数据文件都存放在上一级目录，则可以使用以下方式调整路径：
    # parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
    # data_dir = parent_dir
    # 这里假设 DATA_DIR 指向正确的数据目录
    data_dir = DATA_DIR

    unique_id = "20241108144903"

    # 构造设备列表：字母从 A 到 G，每个字母下的编号为 05, 06, 07, 08
    devices = []
    for letter in "ABCDEFG":
        for num in ["05", "06", "07", "08"]:
            devices.append(letter + num)

    output_lines = []
    for device in devices:
        result = process_device(device, unique_id, data_dir)
        output_lines.append(result)

    # 一次性打印所有结果
    print("\n".join(output_lines))

if __name__ == "__main__":
    main()


=== Device A05 Pilot Signal Measurements ===
Measurement 1 - Pilot phi_P: -0.355483 rad
Measurement 2 - Pilot phi_P: -2.861887 rad

=== Device A05 Loopback Signal Measurements ===
Measurement 1 - Loopback phi_LB: -2.531695 rad
Measurement 2 - Loopback phi_LB: -0.101943 rad
--------------------------------------------------
=== Device A06 Pilot Signal Measurements ===
Measurement 1 - Pilot phi_P: -1.687827 rad
Measurement 2 - Pilot phi_P: -0.275140 rad

=== Device A06 Loopback Signal Measurements ===
Measurement 1 - Loopback phi_LB: -4.107223 rad
Measurement 2 - Loopback phi_LB: -0.830782 rad
--------------------------------------------------
=== Device A07 Pilot Signal Measurements ===
Measurement 1 - Pilot phi_P: -1.935023 rad
Measurement 2 - Pilot phi_P: -1.456983 rad

=== Device A07 Loopback Signal Measurements ===
Measurement 1 - Loopback phi_LB: -0.638810 rad
Measurement 2 - Loopback phi_LB: -0.607733 rad
--------------------------------------------------
=== Device A08 Pilot Sign